In [10]:
import json
import os

path = os.path.join(r"D:\Develop\PythonProjects\pyOTDR\Пионерская 14 [1] - АТС Светлый 1 4 [19]-dump.json")

with open(path) as f:
    data = json.load(f)
    fname = f.name
print(fname.split('\\')[-1])

Пионерская 14 [1] - АТС Светлый 1 4 [19]-dump.json


In [11]:
import re


regexp = r'(.*)\[(.*)\].*[!-](.*)\[(.*)\](.*)'
addrStart, numStart, addrEnd, numEnd = re.findall(regexp, data['filename'], re.IGNORECASE)[0][:-1]

addrStart = addrStart.strip(' ')
addrEnd = addrEnd.strip(' ')

print(addrStart,numStart)
print(addrEnd, numEnd) 


Пионерская 14 1
АТС Светлый 1 4 19


In [12]:
if str(data["FxdParams"]["unit"]) == "km (kilometers)":
    unit = "км"
else:
    unit = "ошибка"

print(f'Начало: {addrStart}')
print(f'Кабель: тип кабеля')
print(f'Диапазон: {data["FxdParams"]["range"]:6.3f} {unit}')
print(f'Длина волны: {data["FxdParams"]["wavelength"]}')
print(f'Порог потерь: {(data["FxdParams"]["loss thr"]).replace("dB","дБ")}')
print('Дата')
print(f'OTDR: {data["SupParams"]["OTDR"]} S/N: {data["SupParams"]["OTDR S/N"]}')
print(f'Модуль: {data["SupParams"]["module"]} S/N: {data["SupParams"]["module S/N"]}')
print('Заказчик: ПАО "Ростелеком')
print('Подрядчик: АО "ТКТ-Строй')
print('Комментарий')



Начало: Пионерская 14
Кабель: тип кабеля
Диапазон: 10.267 км
Длина волны: 1550.0 nm
Порог потерь: 0.100 дБ
Дата
OTDR: AQ1000 S/N: C3UC07069F
Модуль: MODEL_H00_0 S/N: 
Заказчик: ПАО "Ростелеком
Подрядчик: АО "ТКТ-Строй
Комментарий


In [13]:
print(f'Конец: {addrEnd}')
print(f'Волокно: {numStart}')
print(f'Импульс: {(data["FxdParams"]["pulse width"]).replace("ns","нс")}')
print(f'Коэф. преломления: {data["FxdParams"]["index"]}')
print(f'Порог отражения: {data["FxdParams"]["refl thr"]}')
print(f'Файл: {data["filename"]}')

parseName = data['filename']


Конец: АТС Светлый 1 4
Волокно: 1
Импульс: 30 нс
Коэф. преломления: 1.460000
Порог отражения: -65.535 dB
Файл: Пионерская 14 [1] - АТС Светлый 1 4 [19].SOR


In [14]:
print('Результат измерений\n')
numEvents = data["KeyEvents"]["num events"]
distance = data["KeyEvents"][f'event {numEvents}']['distance']
totalLoss = data["KeyEvents"]["Summary"]['total loss']
#distance = data["KeyEvents"][f'even {numEvents}']['distance']
lenghtLoss = float(totalLoss)/float(distance)


print(f'Длина волокна:\t{distance} {unit}')
print(f'Затухание:\t{lenghtLoss:5.3f} дБ/{unit}')
print(f'Полные потери:\t{totalLoss} дБ')


print("\n")


print(f'Таблица событий')
print(f'№\tТип\t\tДистанция\tПотери, дБ\tОтражение, дБ\tЗатухание, дБ/км')


for numEvent in range(numEvents):
    event = data["KeyEvents"][f'event {numEvent+1}']
    spliceLoss = "---" if float(event["splice loss"]) == 0.00 else event["splice loss"]
    reflectLoss = "---"  if event["refl loss"] == "0.000" else event["refl loss"]
        
    if numEvent+1 == numEvents:
        typeEvent = "Конец"
    elif float(event["splice loss"]) < 0:
        typeEvent = "Положит. дефект"
    else:
        typeEvent = "Потери"
            
    print(f'{numEvent+1}\t{typeEvent}\t\t{event["distance"]}\t\t{spliceLoss}\t\t{reflectLoss}\t\t{event["slope"]}')



Результат измерений

Длина волокна:	3.334 км
Затухание:	0.276 дБ/км
Полные потери:	0.921 дБ


Таблица событий
№	Тип		Дистанция	Потери, дБ	Отражение, дБ	Затухание, дБ/км
1	Потери		1.442		0.164		---		0.182
2	Конец		3.334		---		-66.471		0.265
